In [1]:
import pandas as pd
import sqlalchemy as sa  # Corrigido
from sqlalchemy import create_engine  # Importação correta para criar o engine
import os
import sys
sys.path.append(r'c:\Projetos\Python\JornadaDeDados')

pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv("microdados_enem_2023/DADOS/MICRODADOS_ENEM_2023.csv", delimiter=";", encoding="latin-1")

In [3]:
df.columns.values

array(['NU_INSCRICAO', 'NU_ANO', 'TP_FAIXA_ETARIA', 'TP_SEXO',
       'TP_ESTADO_CIVIL', 'TP_COR_RACA', 'TP_NACIONALIDADE',
       'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'TP_ENSINO',
       'IN_TREINEIRO', 'CO_MUNICIPIO_ESC', 'NO_MUNICIPIO_ESC',
       'CO_UF_ESC', 'SG_UF_ESC', 'TP_DEPENDENCIA_ADM_ESC',
       'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC', 'CO_MUNICIPIO_PROVA',
       'NO_MUNICIPIO_PROVA', 'CO_UF_PROVA', 'SG_UF_PROVA',
       'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC',
       'TP_PRESENCA_MT', 'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC',
       'CO_PROVA_MT', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC',
       'NU_NOTA_MT', 'TX_RESPOSTAS_CN', 'TX_RESPOSTAS_CH',
       'TX_RESPOSTAS_LC', 'TX_RESPOSTAS_MT', 'TP_LINGUA',
       'TX_GABARITO_CN', 'TX_GABARITO_CH', 'TX_GABARITO_LC',
       'TX_GABARITO_MT', 'TP_STATUS_REDACAO', 'NU_NOTA_COMP1',
       'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5',
       'NU_NOTA_REDACAO', 'Q001', 'Q002', '

In [4]:
colunasDadosQstSocioEconomico = ['NU_INSCRICAO','Q001', 'Q002', 'Q003', 'Q004', 'Q005', 'Q006',
       'Q007', 'Q008', 'Q009', 'Q010', 'Q011', 'Q012', 'Q013', 'Q014',
       'Q015', 'Q016', 'Q017', 'Q018', 'Q019', 'Q020', 'Q021', 'Q022',
       'Q023', 'Q024', 'Q025']

In [5]:
#Aqui estamos separando os dados pelas colunas selecionadas no objeto colunasDadosProvaObj
dadosQstSocioEconTratamento = df.filter(items=colunasDadosQstSocioEconomico)

In [6]:
dadosQstSocioEconTratamento.head().query('NU_INSCRICAO == 210059085136')

,NU_INSCRICAO,Q001,Q002,Q003,Q004,Q005,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,210059085136,A,F,E,D,5,F,C,C,D,C,D,C,B,B,D,C,C,B,B,A,B,B,A,A,B


In [7]:
dadosUnpivot = dadosQstSocioEconTratamento.melt(
    id_vars=['NU_INSCRICAO'],  # Colunas que serão mantidas fixas
    var_name='Questoes',       # Nome da coluna que conterá as questões
    value_name='Resposta'      # Nome da coluna que conterá as respostas
)

In [8]:
# Exibe o DataFrame resultante
dadosUnpivot.dtypes

NU_INSCRICAO     int64
Questoes        object
Resposta        object
dtype: object

In [9]:
dadosUnpivot.query('NU_INSCRICAO == 210059085136')

,NU_INSCRICAO,Questoes,Resposta
0,210059085136,Q001,A
3933955,210059085136,Q002,F
7867910,210059085136,Q003,E
11801865,210059085136,Q004,D
15735820,210059085136,Q005,5
19669775,210059085136,Q006,F
23603730,210059085136,Q007,C
27537685,210059085136,Q008,C
31471640,210059085136,Q009,D
35405595,210059085136,Q010,C


In [10]:
dadosUnpivot.head()

,NU_INSCRICAO,Questoes,Resposta
0,210059085136,Q001,A
1,210059527735,Q001,F
2,210061103945,Q001,H
3,210060214087,Q001,D
4,210059980948,Q001,B


In [11]:
tabela = 'RespostaQstSocEconomico'

tipo_dado_map = {
    'int64': 'BIGINT',
    'float64': 'FLOAT',
    'object': 'VARCHAR({})',
    'bool': 'BIT',
    'datetime64[ns]': 'DATETIME'
}

def calcular_max_varchar_colunas(dadosUnpivot, coluna):
    
    #retornar o comprimento máximo da coluna
    if dadosUnpivot[coluna].dtype == 'object':
        return dadosUnpivot[coluna].apply(lambda x: len(str(x))).max()
    return None


def gerar_sql_create_table(dadosUnpivot, tabela):
    colunas = []
    
    for coluna, tipo in dadosUnpivot.dtypes.items():
        if str(tipo) in tipo_dado_map:
            tipo_sql = tipo_dado_map[str(tipo)]
            if 'VARCHAR' in tipo_sql:
                max_length = calcular_max_varchar_colunas(dadosUnpivot, coluna)
                if max_length is None:
                    max_length = 255  # Valor padrão caso não seja possível calcular
                tipo_sql = tipo_sql.format(max_length)
            colunas.append(f"[{coluna}] {tipo_sql}")
        else:
            colunas.append(f"[{coluna}] VARCHAR(MAX)")  # Tipo padrão para casos não mapeados
    
    sql_create = f"CREATE TABLE {tabela} (\n" + ",\n".join(colunas) + "\n);"
    return sql_create

# Gerando o script SQL
script_sql_create = gerar_sql_create_table(dadosUnpivot, tabela)

# Exibe o script SQL, basta apenas copiar o result e colar no SSMS que será criado a tabela
print(script_sql_create)

CREATE TABLE RespostaQstSocEconomico (
[NU_INSCRICAO] BIGINT,
[Questoes] VARCHAR(4),
[Resposta] VARCHAR(2)
);


In [12]:
#Etapa que insere os dados no banco de dados

from tqdm import tqdm

#from sqlalchemy import create_engine
from app.db_config import database_config

# Construa a string de conexão corretamente formatada
connection_string = (
    f"mssql+pyodbc://{database_config['username']}:{database_config['password']}"
    f"@{database_config['host']}/{database_config['database']}"
    "?driver=ODBC+Driver+17+for+SQL+Server"
)

# Crie o engine do SQLAlchemy
engine = create_engine(connection_string)

chunk_size = 100000  # Defina o tamanho do chunk conforme necessário
total_chunks = (len(dadosUnpivot) // chunk_size) + 1  # Calcula o número total de chunks

try:
    with tqdm(total=total_chunks, desc="Inserindo dados no banco") as pbar:
        for start in range(0, len(dadosUnpivot), chunk_size):
            end = min(start + chunk_size, len(dadosUnpivot))
            chunk = dadosUnpivot.iloc[start:end]
            chunk.to_sql('RespostaQstSocEconomico', con=engine, if_exists='append', index=False)
            pbar.update(1)  # Atualiza a barra de progresso
    print("Dados inseridos com sucesso!")
except Exception as e:
    print("Erro ao inserir os dados:", e)

Inserindo dados no banco: 100%|██████████| 984/984 [10:32<00:00,  1.56it/s]

Dados inseridos com sucesso!
